<a href="https://colab.research.google.com/github/ManelSoengas/NLP_Curs/blob/main/Utilitzant_Transformers_NLP_Tasques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NLP Tasques**

---
Es treballarà amb les següents tasques lingüístiques comunes que són essencials per treballar tant amb els models tradicionals de NLP com amb els LLM moderns:

1. Classificació de fitxes
1. Modelatge de llenguatge emmascarat (com BERT)
1. Resum
1. Traducció
1. Formació prèvia al modelatge del llenguatge causal (com ara GPT-2)
1. Resposta a preguntes

Aquestes tasques fonamentals constitueixen la base del funcionament dels grans models lingüístics (LLM) i comprendre'ls és crucial per treballar eficaçment amb els models lingüístics més avançats actuals.


# **Token Classificació**

---

És una tasca de processament del llenguatge natural (NLP) que consisteix a assignar una etiqueta específica a cada token (paraula o subparaula) dins d’un text.


1. **Named Entity Recognition (NER)**: Identificar entitats com noms de persona, llocs o organitzacions.


Text: "Albert viu a Barcelona i treballa a Google."
Labels identificades:

* Albert → PER (persona)

* Barcelona → LOC (lloc)

* Google → ORG (organització)

1. **Part-of-Speech Tagging (POS)**: Assignar la categoria gramatical a cada paraula.

Text: "El gat menja peix."
Labelsidentificades:

* El → DET

* gat → NOUN

* menja → VERB

* peix → NOUN

1. **Chunking o Phrase Detection**: Agrupar tokens en estructures sintàctiques com sintagmes nominals.

"El gat blanc" → tots els tokens poden ser etiquetats com a B-NP, I-NP, I-NP (NP = Noun Phrase)

**Preparing the data**

In [2]:
! pip install datasets


from datasets import load_dataset

raw_datasets = load_dataset("conll2003")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [9]:
# Una ullada al contingut del datset

raw_datasets["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [10]:
raw_datasets["train"][0]["ner_tags"]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

Aquestes són les etiquetes com a nombres enters preparats per a l'entrenament, però no són necessàriament útils quan volem inspeccionar les dades. Igual que per a la classificació de text, podem accedir a la correspondència entre aquests nombres enters i els noms d'etiquetes mirant l'atribut de característiques del nostre conjunt de dades:

In [11]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [12]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

1. O significa que la paraula no correspon a cap entitat.
1. B-PER/I-PER significa que la paraula correspon al començament de/es troba dins d'una entitat personal.
1. B-ORG/I-ORG significa que la paraula correspon al començament de/es troba dins d'una entitat de l'organització.
1. B-LOC/I-LOC significa que la paraula correspon al començament de/es troba dins d'una entitat d'ubicació.
1. B-MISC/I-MISC significa que la paraula correspon al començament de/es troba dins d'una entitat diversa.

Ara descodificant les etiquetes que vam veure anteriorment ens dóna això:

In [13]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


**Processant les dades**

---



Com és habitual, els nostres textos s'han de convertir en identificadors de testimoni abans que el model els tingui sentit. Una gran diferència en el cas de les tasques de classificació de testimonis és que tenim entrades prèviament tokenitzades. Afortunadament, l'API de tokenizer pot fer-ho amb força facilitat; només hem d'avisar el tokenitzador amb una bandera especial.

Per començar, creem el nostre objecte tokenizer. Com hem dit abans, utilitzarem un model preentrenat de BERT, així que començarem baixant i guardant a la memòria cau el tokenizer associat:

In [14]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [15]:
tokenizer.is_fast

True

In [16]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

Como podemos ver, el tokenizador añadió los tokens especiales utilizados por el modelo ([CLS] al principio y [SEP] al final) y dejó la mayoría de las palabras intactas. Sin embargo, la palabra "lamb" se tokenizó en dos subpalabras: la y ##mb. Esto introduce una discrepancia entre nuestras entradas y las etiquetas: la lista de etiquetas solo tiene 9 elementos, mientras que nuestra entrada ahora tiene 12 tokens. Contabilizar los tokens especiales es fácil (sabemos que están al principio y al final), pero también debemos asegurarnos de alinear todas las etiquetas con las palabras correctas.

Por suerte, al usar un tokenizador rápido, tenemos acceso a los superpoderes de los tokenizadores, lo que significa que podemos asignar fácilmente cada token a su palabra correspondiente.

In [17]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

Amb una mica de treball, podem ampliar la nostra llista d'etiquetes perquè coincideixi amb les fitxes. La primera regla que aplicarem és que les fitxes especials reben una etiqueta de -100. Això es deu al fet que per defecte -100 és un índex que s'ignora a la funció de pèrdua que utilitzarem (entropia creuada). Aleshores, cada testimoni rep la mateixa etiqueta que el testimoni que va iniciar la paraula que hi ha dins, ja que formen part de la mateixa entitat. Per a fitxes dins d'una paraula però no al principi, substituïm la B- per I- (ja que la fitxa no comença l'entitat):

In [18]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [19]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


Per preprocessar tot el nostre conjunt de dades, hem de tokenitzar totes les entrades i aplicar align_labels_with_tokens() a totes les etiquetes. Per aprofitar la velocitat del nostre tokenitzador ràpid, el millor és tokenitzar molts textos al mateix temps, així que escriurem una funció que processi una llista d'exemples i utilitzarem el mètode Dataset.map() amb l'opció batched=True. L'única cosa que és diferent del nostre exemple anterior és que la funció word_ids() necessita obtenir l'índex de l'exemple del qual volem els ID de paraula quan les entrades al tokenizer són llistes de textos (o en el nostre cas, llista de llistes de paraules), així que també ho afegim:

In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [21]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

**Fine-tuning el model amb Keras**

---
El codi real que utilitza Keras serà molt semblant a l'anterior; els únics canvis són la manera com les dades s'agrupen en un lot i la funció de càlcul mètric.

**Quin és el problema?**

Quan preprocesses un conjunt de dades per a models com BERT o RoBERTa:

1. Les seqüències d'entrada (tokens) sovint no tenen la mateixa longitud.

1. Això es resol fent padding: afegint tokens especials fins que totes les seqüències tenen la mateixa llargada.

1. Però en la classificació de tokens, també tens etiquetes (labels) per a cada token, i cal fer padding també d’aquestes etiquetes.

**Però atenció...**

1. No podem fer padding de les etiquetes amb zeros, perquè el model aprendria a predir etiquetes pels tokens de padding (el que no té sentit).

**Una possible solució:**

Es fa padding de les etiquetes amb el valor especial -100. Aquest valor és ignorat pel càlcul de la loss (funció de pèrdua) durant l'entrenament.

**DataCollatorForTokenClassification** és essencial perquè:

1. Fa padding de manera coherent tant per als inputs com per als labels.

1. Evita que el model aprengui de tokens artificials (gràcies al -100).

1. Estalvia feina i errors comuns quan prepares batches de dades per entrenar.


In [22]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer, return_tensors="tf"
)

In [23]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(3)])
batch["labels"]

<tf.Tensor: shape=(3, 12), dtype=int64, numpy=
array([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0,
        -100],
       [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100,
        -100],
       [-100,    5,    6,    6,    6,    0,    0,    0,    0,    0, -100,
        -100]])>

Comparant això amb les etiquetes del primer i segon element del conjunt de dades:

In [24]:
for i in range(3):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[-100, 1, 2, -100]
[-100, 5, 6, 6, 6, 0, 0, 0, 0, 0, -100]


El recopilador de dades ja està a punt! Ara s'utilitza per fer un tf.data.Dataset amb el mètode to_tf_dataset(). També es pot utilitzar el model.prepare_tf_dataset() per fer-ho amb una mica menys de codi normal.

In [25]:
# Objectiu : Convertir aquests datasets Hugging Face en datasets TensorFlow (tf.data.Dataset) per entrenar un model amb TensorFlow/Keras.



tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)

tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

**Definint el model**

---

Com que estem treballant en un problema de classificació de testimonis, utilitzarem la classe TFAutoModelForTokenClassification. El més important a recordar a l'hora de definir aquest model és transmetre informació sobre el nombre d'etiquetes que tenim. La manera més senzilla de fer-ho és passar aquest número amb l'argument num_labels, però si volem que funcioni un estris d'inferència agradable com el que vam veure al principi d'aquesta secció, és millor establir les correspondències d'etiquetes correctes.

S'han d'establir mitjançant dos diccionaris, id2label i label2id, que contenen el mapeig d'ID a etiqueta i viceversa:

In [26]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [27]:
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
model.config.num_labels

9

**Fine-tuning el model**

---

Un model fine-tuned (o model ajustat fi) és un model preentrenat al qual se li ha fet un entrenament addicional sobre una tasca específica i amb un conjunt de dades concret.

Ara estem preparats per entrenar el nostre model! Tanmateix, primer hem de fer una mica més de neteja: hauríem d'iniciar sessió a Hugging Face i definir els nostres hiperparàmetres d'entrenament. Al treballar en un quadern, hi ha una funció de comoditat que us ajudarà amb això:


In [29]:
from huggingface_hub import notebook_login

notebook_login()

In [30]:
from transformers import create_optimizer
import tensorflow as tf

# Train in mixed-precision float16
# Comment this line out if you're using a GPU that will not benefit from this
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_epochs = 3
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

Cal tenir en compte també que no proporcionem un argument de pèrdua per compilar(). Això es deu al fet que els models poden calcular la pèrdua internament: si compileu sense pèrdua i proporcionar les etiquetes al diccionari d'entrada (com fem als nostres conjunts de dades), el model s'entrenarà utilitzant aquesta pèrdua interna, que serà adequada per a la tasca i el tipus de model que hàgiu escollit.

A continuació, es defineix un PushToHubCallback per carregar el nostre model al concentrador durant l'entrenament i ajustar el model amb aquesta devolució de trucada:

In [31]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(output_dir="bert-finetuned-ner", tokenizer=tokenizer)

model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    callbacks=[callback],
    epochs=num_epochs,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/LeNaM/bert-finetuned-ner into local empty directory.


Epoch 1/3
878/878 [==============================] - 237s 242ms/step - loss: 0.1726 - val_loss: 0.0638
Epoch 2/3
878/878 [==============================] - 232s 264ms/step - loss: 0.0472 - val_loss: 0.0564
Epoch 3/3
878/878 [==============================] - 230s 262ms/step - loss: 0.0277 - val_loss: 0.0537


Mentre es produeix l'entrenament, cada vegada que es desa el model (aquí, cada època) es puja al Hub en segon pla. D'aquesta manera, podreu reprendre el vostre entrenament en una altra màquina si cal.

En aquesta etapa, es pot utilitzar el giny d'inferència al Model Hub per provar el vostre model i compartir-lo amb els vostres amics. Heu ajustat correctament un model en una tasca de classificació de testimonis; enhorabona! Però, realment, què bo és el nostre model? Hauríem d'avaluar algunes mètriques per esbrinar.

**Mètriques**

---
El marc tradicional utilitzat per avaluar la predicció de classificació de testimonis és seqeval. Per utilitzar aquesta mètrica, primer hem d'instal·lar la biblioteca seqeval:


In [32]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=d462cc1a6fbba9685dd8d98fdbedb4276735f973843c8c34106e4277d167c286
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [34]:

!pip install evaluate

import evaluate

metric = evaluate.load("seqeval")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00


Aquesta mètrica no es comporta com la precisió estàndard: en realitat prendrà les llistes d'etiquetes com a cadenes, no com a nombres enters, de manera que haurem de descodificar completament les prediccions i les etiquetes abans de passar-les a la mètrica. Vegem com funciona. Primer, obtindrem les etiquetes del nostre primer exemple de formació:

In [35]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

Aleshores podem crear prediccions falses per a aquells només canviant el valor de l'índex 2:

In [36]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'precision': np.float64(1.0),
  'recall': np.float64(0.5),
  'f1': np.float64(0.6666666666666666),
  'number': np.int64(2)},
 'ORG': {'precision': np.float64(1.0),
  'recall': np.float64(1.0),
  'f1': np.float64(1.0),
  'number': np.int64(1)},
 'overall_precision': np.float64(1.0),
 'overall_recall': np.float64(0.6666666666666666),
 'overall_f1': np.float64(0.8),
 'overall_accuracy': 0.8888888888888888}

Això està enviant molta informació! Obtenim la precisió, el record i la puntuació F1 per a cada entitat separada, així com en general. Ara vegem què passa si provem d'utilitzar les nostres prediccions del model real per calcular algunes puntuacions reals.

A TensorFlow no li agrada concatenar les nostres prediccions, perquè tenen longituds de seqüència variables. Això vol dir que no podem utilitzar només model.predict(), però això no ens aturarà. Obtenim algunes prediccions per lot a la vegada i les concatenarem en una gran llista llarga a mesura que avancem, deixant anar els -100 fitxes que indiquen emmascarament/encoixinat, i després calcularem mètriques a la llista al final:

In [38]:
import numpy as np

all_predictions = []
all_labels = []
for batch in tf_eval_dataset:
    logits = model.predict_on_batch(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    for prediction, label in zip(predictions, labels):
        for predicted_idx, label_idx in zip(prediction, label):
            if label_idx == -100:
                continue
            all_predictions.append(label_names[predicted_idx])
            all_labels.append(label_names[label_idx])
metric.compute(predictions=[all_predictions], references=[all_labels])

{'LOC': {'precision': np.float64(0.9589490968801314),
  'recall': np.float64(0.9537289058247143),
  'f1': np.float64(0.9563318777292577),
  'number': np.int64(1837)},
 'MISC': {'precision': np.float64(0.8429003021148036),
  'recall': np.float64(0.9078091106290672),
  'f1': np.float64(0.8741514360313316),
  'number': np.int64(922)},
 'ORG': {'precision': np.float64(0.893740902474527),
  'recall': np.float64(0.9157345264727815),
  'f1': np.float64(0.9046040515653776),
  'number': np.int64(1341)},
 'PER': {'precision': np.float64(0.9533898305084746),
  'recall': np.float64(0.9771986970684039),
  'f1': np.float64(0.96514745308311),
  'number': np.int64(1842)},
 'overall_precision': np.float64(0.9235448865504768),
 'overall_recall': np.float64(0.945304611242006),
 'overall_f1': np.float64(0.9342980705256155),
 'overall_accuracy': 0.9854153175958086}

**Utilitzant el Fined-Tunning model**

---
Per utilitzar-lo localment en una canalització, només heu d'especificar l'identificador de model adequat:




In [39]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "huggingface-course/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


[{'entity_group': 'PER',
  'score': np.float32(0.9988506),
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': np.float32(0.96476245),
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': np.float32(0.9986118),
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]